In [9]:
import numpy as np
import pandas as pd
import os

In [10]:
df = pd.read_excel('kpmg_raw.xlsx', sheet_name="CustomerDemographic")
df.columns = df.iloc[0]
df = df.iloc[1:]
del df['default']
df.insert(6,column='age',value=np.nan)
df.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,NaN,Executive Secretary,Health,Mass Customer,N,Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,NaN,Administrative Officer,Financial Services,Mass Customer,N,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,NaN,Recruiting Manager,Property,Mass Customer,N,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,NaN,IT,Mass Customer,N,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,NaN,Senior Editor,NaN,Affluent Customer,N,Yes,8


In [11]:
print(df.columns)
print(len(df.columns))
print(min(df['customer_id']),max(df['customer_id']))

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'age', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure'],
      dtype='object', name=0)
13
1 4000


In [12]:
df.isnull().sum()

0
customer_id                               0
first_name                                0
last_name                               125
gender                                    0
past_3_years_bike_related_purchases       0
DOB                                      87
age                                    4000
job_title                               506
job_industry_category                   656
wealth_segment                            0
deceased_indicator                        0
owns_car                                  0
tenure                                   87
dtype: int64

In [13]:
print(df['job_title'].value_counts())
print(df['job_industry_category'].value_counts())

Business Systems Development Analyst    45
Tax Accountant                          44
Social Worker                           44
Internal Auditor                        42
Legal Assistant                         41
                                        ..
Programmer Analyst IV                    4
Health Coach III                         3
Health Coach I                           3
Research Assistant III                   3
Developer I                              1
Name: job_title, Length: 195, dtype: int64
Manufacturing         799
Financial Services    774
Health                602
Retail                358
Property              267
IT                    223
Entertainment         136
Argiculture           113
Telecommunications     72
Name: job_industry_category, dtype: int64


In [14]:
import datetime
for i in df.index:
    if pd.isna(df['DOB'].iloc[i-1]):
        continue
    elif type(df['DOB'].iloc[i-1]) == str:
        df['age'].iloc[i-1] = 2020 - int(df['DOB'].iloc[i-1][:4])
    else:
        df['age'].iloc[i-1] = 2020 - df['DOB'].iloc[i-1].year
    if df['age'].iloc[i-1] >= 100:
        print(df.iloc[i-1])

0
customer_id                                           34
first_name                                      Jephthah
last_name                                       Bachmann
gender                                                 U
past_3_years_bike_related_purchases                   59
DOB                                           1843-12-21
age                                                  177
job_title                                Legal Assistant
job_industry_category                                 IT
wealth_segment                         Affluent Customer
deceased_indicator                                     N
owns_car                                              No
tenure                                                20
Name: 34, dtype: object


D:\Programs\conda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
print(df['gender'].unique())

['F' 'Male' 'Female' 'U' 'Femal' 'M']


In [23]:
for i in df.index:
    if df['gender'].iloc[i-1][0] == 'M':
        df['gender'].iloc[i-1] = 'Male'
    elif df['gender'].iloc[i-1][0] == 'F':
        df['gender'].iloc[i-1] = 'Female'
    elif df['gender'].iloc[i-1][0] == 'U':
        df['gender'].iloc[i-1] = 'Unknown'

In [25]:
print(df['gender'].unique())

['Female' 'Male' 'Unknown']


# Data Quality Issues:
* Date of Birth values incorrect
* Lots of missing values
* Inconsistent notation for gender

In [30]:
df['last_name']=df['last_name'].fillna(method='ffill')
df['DOB']=df['DOB'].fillna(method='ffill')
df['age']=df['age'].fillna(method='ffill')
df['job_title']=df['job_title'].fillna(method='bfill')
df['job_industry_category']=df['job_industry_category'].fillna(method='ffill')
df['tenure']=df['tenure'].fillna(df['tenure'].mean())

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
1,1,Laraine,Medendorp,Female,93,1953-10-12 00:00:00,67.0,Executive Secretary,Health,Mass Customer,N,Yes,11.000000
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,40.0,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.000000
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,66.0,Recruiting Manager,Property,Mass Customer,N,Yes,15.000000
4,4,Talbot,Dearle,Male,33,1961-10-03 00:00:00,59.0,Senior Editor,IT,Mass Customer,N,No,7.000000
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,43.0,Senior Editor,IT,Affluent Customer,N,Yes,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,3996,Rosalia,Halgarth,Female,8,1975-08-09 00:00:00,45.0,VP Product Management,Health,Mass Customer,N,No,19.000000
3997,3997,Blanch,Nisuis,Female,87,2001-07-13 00:00:00,19.0,Statistician II,Manufacturing,High Net Worth,N,Yes,1.000000
3998,3998,Sarene,Woolley,Unknown,60,2001-07-13 00:00:00,19.0,Assistant Manager,IT,High Net Worth,N,No,10.657041
3999,3999,Patrizius,Woolley,Male,11,1973-10-24 00:00:00,47.0,Software Engineer IV,Manufacturing,Affluent Customer,N,Yes,10.000000


In [31]:
print(df.columns)

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'age', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure'],
      dtype='object', name=0)


In [29]:
submission = pd.DataFrame({
        "customer_id": df["customer_id"],
        "first_name": df["first_name"],
        "last_name": df["last_name"],
        "gender": df["gender"],
        "past_3_years_bike_related_purchases": df["past_3_years_bike_related_purchases"],
        "DOB": df["DOB"],
        "age": df["age"],
        "job_title": df["job_title"],
        "job_industry_category": df["job_industry_category"],
        "wealth_segment": df["wealth_segment"],
        "deceased_indicator": df["deceased_indicator"],
        "owns_car": df["owns_car"],
        "tenure": df["tenure"]
    })
submission.to_excel('updated_cd.xlsx', index=False)
submission.head(10)

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,age,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure
1,1,Laraine,Medendorp,Female,93,1953-10-12 00:00:00,67.0,Executive Secretary,Health,Mass Customer,N,Yes,11.0
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,40.0,Administrative Officer,Financial Services,Mass Customer,N,Yes,16.0
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,66.0,Recruiting Manager,Property,Mass Customer,N,Yes,15.0
4,4,Talbot,Dearle,Male,33,1961-10-03 00:00:00,59.0,Senior Editor,IT,Mass Customer,N,No,7.0
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,43.0,Senior Editor,IT,Affluent Customer,N,Yes,8.0
6,6,Curr,Duckhouse,Male,35,1966-09-16 00:00:00,54.0,Media Manager I,Retail,High Net Worth,N,Yes,13.0
7,7,Fina,Merali,Female,6,1976-02-23 00:00:00,44.0,Media Manager I,Financial Services,Affluent Customer,N,Yes,11.0
8,8,Rod,Inder,Male,31,1962-03-30 00:00:00,58.0,Media Manager I,Financial Services,Mass Customer,N,No,7.0
9,9,Mala,Lind,Female,97,1973-03-10 00:00:00,47.0,Business Systems Development Analyst,Argiculture,Affluent Customer,N,Yes,8.0
10,10,Fiorenze,Birdall,Female,49,1988-10-11 00:00:00,32.0,Senior Quality Engineer,Financial Services,Mass Customer,N,Yes,20.0
